月球岩石對於科學探索，以及了解我們的宇宙與地球十分重要。 其可以告訴我們行星及衛星如何形成，並在我們準備進一步探索太空時給予指引。

在人類登陸月球的六次阿波羅任務，以及由機器人探測器進行登陸的三次月球任務中，我們已成功帶回 383 公斤 (超過 800 磅) 的岩石、土壤，以及岩芯樣本。 
所有以上樣本皆已分類並拍照存檔，而且可以在 NASA Lunar Sample and Photo Catalog (NASA 月球樣本及相片目錄) 上觀看。

這項挑戰因為兩個原因而特別困難：

我們無法隨意將太空人送上月球，以收集更多特定類型的岩石。
當太空人再次登陸月球時，他們必須知道哪些類型的岩石樣本，以及需要收集多少個岩石樣本，才能在地球上重新補貨集合。
地球上的專家會對月球岩石樣本進行全面性的分析及清理，而鑒藏人員則能了解其所擁有的樣本有哪些、會收到什麼樣本的請求，以及確保研究能夠繼續進行的最具挑戰性部分為何。

本單元開始探索如何使用資料和一些 Python 編碼來了解我們所擁有的內容。 從這項分析中，您可以針對下一組太空人在 2025 年登陸月球時應該尋找的專案提出建議。

=============================================================================================================

In [1]:
import pandas as pd
rock_samples = pd.read_csv("rocksamples.csv")

In [2]:
rock_samples.head()

,ID,Mission,Type,Subtype,Weight (g),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,125.8,88.36
1,10002,Apollo11,Soil,Unsieved,5629.0,93.73
2,10003,Apollo11,Basalt,Ilmenite,213.0,65.56
3,10004,Apollo11,Core,Unsieved,44.8,71.76
4,10005,Apollo11,Core,Unsieved,53.4,40.31


In [3]:
rock_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            2229 non-null   int64  
 1   Mission       2229 non-null   object 
 2   Type          2229 non-null   object 
 3   Subtype       2226 non-null   object 
 4   Weight (g)    2229 non-null   float64
 5   Pristine (%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


在此輸出中，我們可看到在阿波羅任務中所收集到的 2229 組樣本。 觀察資料範例，我們可看到每一列都包含：

- 識別碼 - 用來在 NASA 追蹤樣本的唯一識別碼。
- 任務 - 負責取得樣本的任務。
- 類型 - 樣本的類型 (岩石的類型或其他分類)。
- 子類型 - 更具體的類型分類。
- 重量 (公克) - 樣本的原始重量，以公克為單位。
- 完整度 (%) - 樣本的剩餘百分比 (部分樣本會在研究中消耗)。

=============================================================================================================

### 轉換樣本重量

準備岩石樣本資料以供稍後的計算，我們必須了解火箭重量通常是以公斤測量，而不是公克。 

因此，我們需要將原始岩石樣本重量從公克數轉換成公斤數，以便稍後進行資料分析。

在這裡，我們要先修改 [重量 (公克)] 資料行中的值，將該值乘以 0.001。 接著將該資料行變更為更精確的名稱，亦即重量 (公斤)。

In [4]:
rock_samples['Weight (g)'] = rock_samples['Weight (g)'].apply(lambda x : x*0.001)
rock_samples.rename(columns={'Weight (g)' : 'Weight (kg)'}, inplace=True)
rock_samples.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56
3,10004,Apollo11,Core,Unsieved,0.0448,71.76
4,10005,Apollo11,Core,Unsieved,0.0534,40.31


### 建立新的 DataFrame
建立稱為 missions 的新 DataFrame

作為在六次阿波羅任務中所帶回樣本的資料摘要。 

在此 DataFrame 中建立稱為任務的資料行，並使每個任務都有一個資料列。

In [21]:
missions = pd.DataFrame()
missions['Mission'] = rock_samples['Mission'].unique()
#去除數組中的重複數字，並進行排序後輸出

missions.head()

,Mission
0,Apollo11
1,Apollo12
2,Apollo14
3,Apollo15
4,Apollo16


In [6]:
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Mission  6 non-null      object
dtypes: object(1)
memory usage: 180.0+ bytes


### 每次任務的總樣本總重量
現在您可將資料行新增至 missions DataFrame，以表示在該任務中所收集的樣本總和

In [7]:
sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum()
#使用groupby方法，將rock_samples資料框按照 Mission列進行分組。對每一組計算Weight (kg)列的總和

missions = pd.merge(missions, sample_total_weight, on='Mission')
#將missions資料框和sample_total_weight資料框進行合併。這是基於Mission列進行合併

missions.rename(columns={'Weight (kg)' : 'Sample weight (kg)'}, inplace=True)
missions

,Mission,Sample weight (kg)
0,Apollo11,21.55424
1,Apollo12,34.34238
2,Apollo14,41.83363
3,Apollo15,75.39910
4,Apollo16,92.46262
5,Apollo17,109.44402


### 取得任務之間的重量差異
我們不是火箭專家，請務必查看能夠取得的各層面資料。 
在此案例中，我們可看到樣本總重量隨著每次任務增加，但是很難立刻知道增加的重量。 

我們可將一或多個資料行新增至 missions DataFrame，其只會抓取目前資料列與前一個資料列之間的差異：

In [10]:
missions['Weight diff'] = missions['Sample weight (kg)'].diff()
missions

,Mission,Sample weight (kg),Weight diff
0,Apollo11,21.55424,NaN
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


在第一個資料列中，針對 Apollo11，[重量差異] 資料行中的值為 NaN。 

因為 Apollo11 是第一次任務，所以對於 Apollo11 所收集的岩石重量，並不存在與前一次任務的差異。 
我們可以使用 0 來填入此 NaN 值

In [11]:
missions['Weight diff'] = missions['Weight diff'].fillna(value=0)
missions

,Mission,Sample weight (kg),Weight diff
0,Apollo11,21.55424,0.00000
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


===========================================================================================================

Apollo 計畫中使用的 Saturn V 火箭又稱為三節火箭。 「三節」表示該火箭具有三個部分，每個部分會在不同的時間點燃，以達成不同的目標。

第一節是讓火箭能夠升空約 68 公里的主要推力部分，該部分會墜回地球，以降低火箭的重量。 
接下來，第二節會點燃其引擎，直到火箭幾乎到達地球的軌道，然後該部分也會墜回地球。 
最後一節會讓太空船到達地球軌道，並朝著月球的方向推進。

### 新增指揮艙與登月小艇的資料
透過美國國家航空暨太空總署空間科學數據中心，我們取得了每次任務所使用的每個艙室相關資訊。 
如同您建立樣本資料表時的步驟，建立六個新的資料行，其中三個用於登月小艇，另外三個用於指揮艙：

- 模組名稱
- 艙室質量
- 艙室質量差異

使用 0 填入所有 NaN 值：

In [15]:
missions['Lunar module (LM)'] = ['Eagle (LM-5)', 'Intrepid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)']
missions['LM mass (kg)'] = [15103, 15235, 15264, 16430, 16445, 16456]
missions['LM mass diff'] = missions['LM mass (kg)'].diff()
missions['LM mass diff'] = missions['LM mass diff'].fillna(value=0)

missions['Command module (CM)'] = ['Columbia (CSM-107)', 'Yankee Clipper (CM-108)', 'Kitty Hawk (CM-110)', 'Endeavor (CM-112)', 'Casper (CM-113)', 'America (CM-114)']
missions['CM mass (kg)'] = [5560, 5609, 5758, 5875, 5840, 5960]
missions['CM mass diff'] = missions['CM mass (kg)'].diff()
missions['CM mass diff'] = missions['CM mass diff'].fillna(value=0)

missions

,Mission,Sample weight (kg),Weight diff,Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff
0,Apollo11,21.55424,0.00000,Eagle (LM-5),15103,0.0,Columbia (CSM-107),5560,0.0
1,Apollo12,34.34238,12.78814,Intrepid (LM-6),15235,132.0,Yankee Clipper (CM-108),5609,49.0
2,Apollo14,41.83363,7.49125,Antares (LM-8),15264,29.0,Kitty Hawk (CM-110),5758,149.0
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,1166.0,Endeavor (CM-112),5875,117.0
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,15.0,Casper (CM-113),5840,-35.0
5,Apollo17,109.44402,16.98140,Challenger (LM-12),16456,11.0,America (CM-114),5960,120.0


我們可在每次任務的指揮艙與登月小艇中，新增一些總計：

In [16]:
missions['Total weight (kg)'] = missions['LM mass (kg)'] + missions['CM mass (kg)']
missions['Total weight diff'] = missions['LM mass diff'] + missions['CM mass diff']
missions

,Mission,Sample weight (kg),Weight diff,Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff,Total weight (kg),Total weight diff
0,Apollo11,21.55424,0.00000,Eagle (LM-5),15103,0.0,Columbia (CSM-107),5560,0.0,20663,0.0
1,Apollo12,34.34238,12.78814,Intrepid (LM-6),15235,132.0,Yankee Clipper (CM-108),5609,49.0,20844,181.0
2,Apollo14,41.83363,7.49125,Antares (LM-8),15264,29.0,Kitty Hawk (CM-110),5758,149.0,21022,178.0
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,1166.0,Endeavor (CM-112),5875,117.0,22305,1283.0
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,15.0,Casper (CM-113),5840,-35.0,22285,-20.0
5,Apollo17,109.44402,16.98140,Challenger (LM-12),16456,11.0,America (CM-114),5960,120.0,22416,131.0


=================================================================================================================================

### 比較資料
在預測每次阿提米絲任務可帶回多少樣本時最有趣的一點，就是我們還不知道阿提米絲任務打算使用的太空船完整規格。 
在使用來自太空發射系統 (SLS) 及 Orion 小艇的 NASA 資料頁的部分資訊時，我們可得到重量及載荷的相關資料。

承載重量基本上是指火箭在突破大氣層進入太空時可載運的總重量。 因為決定載荷可能會影響其他各項設計決策，所以載荷數字很可能會比每個小艇的確切重量更精確。

我們知道 Saturn V 載荷是 43,500 公斤，而小艇的重量則依任務而有所不同。 因此，為了判斷可讓我們對 Artemis 任務做出預測的比率，我們可以使用：

- Saturn V 載荷
- 任務樣本重量
- 任務小艇重量

In [18]:
saturnVPayload = 43500
missions['Crewed area : Payload'] = missions['Total weight (kg)'] / saturnVPayload
missions['Sample : Crewed area'] = missions['Sample weight (kg)'] / missions['Total weight (kg)']
missions['Sample : Payload'] = missions['Sample weight (kg)'] / saturnVPayload
missions

,Mission,Sample weight (kg),Weight diff,Lunar module (LM),LM mass (kg),LM mass diff,Command module (CM),CM mass (kg),CM mass diff,Total weight (kg),Total weight diff,Crewed area : Payload,Sample : Crewed area,Sample : Payload
0,Apollo11,21.55424,0.00000,Eagle (LM-5),15103,0.0,Columbia (CSM-107),5560,0.0,20663,0.0,0.475011,0.001043,0.000495
1,Apollo12,34.34238,12.78814,Intrepid (LM-6),15235,132.0,Yankee Clipper (CM-108),5609,49.0,20844,181.0,0.479172,0.001648,0.000789
2,Apollo14,41.83363,7.49125,Antares (LM-8),15264,29.0,Kitty Hawk (CM-110),5758,149.0,21022,178.0,0.483264,0.001990,0.000962
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,1166.0,Endeavor (CM-112),5875,117.0,22305,1283.0,0.512759,0.003380,0.001733
4,Apollo16,92.46262,17.06352,Orion (LM-11),16445,15.0,Casper (CM-113),5840,-35.0,22285,-20.0,0.512299,0.004149,0.002126
5,Apollo17,109.44402,16.98140,Challenger (LM-12),16456,11.0,America (CM-114),5960,120.0,22416,131.0,0.515310,0.004882,0.002516


### 儲存這些比率
接著，我們可使用 mean() 函式，在所有任務之間取得所有這些比率的平均。

In [20]:
crewedArea_payload_ratio = missions['Crewed area : Payload'].mean()
sample_crewedArea_ratio = missions['Sample : Crewed area'].mean()
sample_payload_ratio = missions['Sample : Payload'].mean()

print(crewedArea_payload_ratio)
print(sample_crewedArea_ratio)
print(sample_payload_ratio)

0.4963026819923371
0.002848764392685611
0.0014369195019157087


=================================================================================================